In [1]:
import pandas as pd
import os
import warnings
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
# Ignore specific warnings
warnings.filterwarnings('ignore')
 

In [2]:
# Set environment variable for OpenMP threads
os.environ["OMP_NUM_THREADS"] = "3"

In [3]:
# Directory containing the Excel files
excel_dir = r'C:\Users\hparnell\Desktop\MH10010\Resources'

In [4]:
# Read all Excel files into a list of data frames
data_frames = []
file_names = []
 

In [ ]:
for file in os.listdir(excel_dir):
    if file.endswith('.xlsx') or file.endswith('.xls'):
        df = pd.read_excel(os.path.join(excel_dir, file))
        data_frames.append(df)
        file_names.append(file)

In [ ]:
# Convert DataFrames to text data for clustering
text_data = []
for df in data_frames:
    # Convert each column to string type individually to avoid NULLs due to type mismatches
    for column in df.columns:
        df[column] = df[column].astype(str, errors='ignore')

    #Flatten the DataFrame
    text = df.astype(str).values.flatten()
    text = ' '.join(text)
    text_data.append(text)

In [ ]:
# Preprocessing and vectorization
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(text_data)
 

In [ ]:
# Calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(X)

In [ ]:
# Perform K-means clustering on the similarity matrix
num_clusters = 10  # Adjust based on your needs
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(similarity_matrix)

In [ ]:
# Get cluster labels
labels = kmeans.labels_

In [ ]:
# Create a dictionary to hold the subsets
subsets = {i: [] for i in range(num_clusters)}

In [ ]:
# Assign files to clusters
for i, label in enumerate(labels):
    subsets[label].append(file_names[i])

In [ ]:
# Save each cluster's filenames to a CSV file
output_dir = r'C:\Users\hparnell\Desktop\MH10010\Clustered_Files'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
for cluster, files in subsets.items():
    df = pd.DataFrame(files, columns=["File Name"])
    file_path = os.path.join(output_dir, f'cluster_{cluster}_files.csv')
    df.to_csv(file_path, index=False)
    print(f"Cluster {cluster} DataFrame saved to {file_path}")

In [ ]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("ExcelProcessing") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.13.5") \
    .getOrCreate()

In [ ]:
def get_string_schema(columns):
    return StructType([StructField(column, StringType(), True) for column in columns])

In [ ]:
def load_data_to_dataframe(file_path, schema):
    try:
        df = spark.read.format('com.crealytics.spark.excel') \
                       .option('dataAddress', "'Sheet1'!A1") \
                       .option('header', 'true') \
                       .option('inferSchema', 'false') \
                       .schema(schema) \
                       .load(file_path)
        return df
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

In [ ]:
# Load data for each cluster
dataframes = {i: [] for i in range(num_clusters)}
for cluster_id, files in subsets.items():
    if files:
        # Use the first file to infer the schema
        sample_df = pd.read_excel(os.path.join(excel_dir, files[0]), nrows=0)
        schema = get_string_schema(sample_df.columns)
        for file in files:
            file_path = os.path.join(excel_dir, file)
            df = load_data_to_dataframe(file_path, schema)
            if df:
                dataframes[cluster_id].append(df)

In [ ]:
# Combine all DataFrames into one for each cluster
combined_dfs = {}
for cluster_id, dfs in dataframes.items():
    if dfs:
        combined_df = dfs[0]
        for df in dfs[1:]:
            combined_df = combined_df.union(df)
        combined_dfs[cluster_id] = combined_df

In [ ]:
# Show combined DataFrames for each cluster
for cluster_id, df in combined_dfs.items():
    print(f"Cluster {cluster_id} Combined DataFrame:")
    df.show()